In [1]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [2]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n#!python3 -m pip install pandas\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [3]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [4]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [5]:
!pip install db-dtypes
query_unemployment = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data` 
"""
query = bigquery_client.query(query_unemployment)
df_unemployment = query.to_dataframe()
#df_unemployment.head()

In [6]:
query_wage = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""
query = bigquery_client.query(query_wage)
df_wage = query.to_dataframe()
#df_wage.head()

In [7]:
query_pred_list = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""
query = bigquery_client.query(query_pred_list)
df_pred_list = query.to_dataframe()
#df_pred_list.head()

In [8]:
query_schema = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query = bigquery_client.query(query_schema)
df_schema = query.to_dataframe()
#df_schema.head()

In [9]:
df_three_col = df_unemployment[["uu_id", "week_number", "total_claims"]]
#df_three_col.shape

In [10]:
df_three_col = df_three_col.drop_duplicates()
#print(df_three_col.shape)

In [11]:
df_three_col.sort_values(by=['uu_id', "week_number"],inplace=True)
#df_three_col.head(21)
#df_three_col.tail(3)

In [12]:
print (df_three_col['week_number'].drop_duplicates())

1877     31
1873     33
14073    34
7254     35
1872     36
1871     37
10905     1
10906     2
10910     3
8603      5
8604      6
10907     7
3813      8
10902     9
8602     10
3350     14
3814     16
10903    19
13455    20
8601     24
13456    25
11905    26
10901    27
3812     28
3816     29
11904    30
3811     32
16345    11
3349     13
10894    15
13441    22
5265     17
4818     12
4812     18
4811     21
Name: week_number, dtype: Int64


In [13]:
df_three_col = df_three_col.drop_duplicates()
#print(df_three_col.shape)

In [14]:
df_three_col.sort_values(by=['uu_id', "week_number"],inplace=True)
df_three_col.head(21)
#df_three_col.tail(3)

,uu_id,week_number,total_claims
1877,001cd9ae23064d7f0fd3cd327c873d8d,31,34
1873,001cd9ae23064d7f0fd3cd327c873d8d,33,10
14073,001cd9ae23064d7f0fd3cd327c873d8d,34,25
7254,001cd9ae23064d7f0fd3cd327c873d8d,35,10
1872,001cd9ae23064d7f0fd3cd327c873d8d,36,10
1871,001cd9ae23064d7f0fd3cd327c873d8d,37,13
10905,005be9532fd717dc36d4be318fd9ad25,1,20
10906,005be9532fd717dc36d4be318fd9ad25,2,19
10910,005be9532fd717dc36d4be318fd9ad25,3,19
8603,005be9532fd717dc36d4be318fd9ad25,5,15


In [15]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [16]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count++
            sum += test_data[test_data['week_number].isin(week_id) ]].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        y_pred = test_data[test_data['week_number].isin(max_week) ]].values(2)
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

SyntaxError: invalid syntax (3103802953.py, line 11)

In [17]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [18]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [19]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count++
            sum += test_data[test_data['week_number].isin(week_id) ]].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        y_pred = test_data[test_data['week_number'.isin(max_week) ]].values(2)
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

SyntaxError: invalid syntax (2509376314.py, line 11)

In [20]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [21]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [22]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count++
            sum += test_data[test_data['week_number'.isin(week_id) ]].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        y_pred = test_data[test_data['week_number'.isin(max_week) ]].values(2)
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

SyntaxError: invalid syntax (3662261505.py, line 11)

In [23]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [24]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [25]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count+=1
            sum += test_data[test_data['week_number'.isin(week_id) ]].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        y_pred = test_data[test_data['week_number'.isin(max_week) ]].values(2)
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

AttributeError: 'str' object has no attribute 'isin'

In [26]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [27]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [28]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count+=1
            sum += test_data[test_data['week_number'].isin(week_id) ]].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        y_pred = test_data[test_data['week_number'.isin(max_week) ]].values(2)
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

SyntaxError: unmatched ']' (2085152070.py, line 12)

In [29]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [30]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [31]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count+=1
            sum += test_data[test_data['week_number'].isin(week_id) ].values(2)
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        tmp_row = test_data[test_data['week_number'].isin(max_week) ].values(2)
        #y_pre
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [int]

In [32]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [33]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [34]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count+=1
            tmp_row = test_data[test_data['week_number'] == week_id  ]
            sum+= tmp_row['total_claims']
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        tmp_row = test_data[test_data['week_number'] ==  max_week  ] 
        y_pre = tmp_row['total_claims']
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [35]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)

In [36]:
res = pd.DataFrame(columns = ['uu_id', 'total_claims', 'week_number'])

In [37]:
for cur_uu_id in df_pred_list['uu_id']:
    #print(uu_id)
    test_data = df_three_col[df_three_col["uu_id"].isin([cur_uu_id]) ]
    #test_data = test_data.tail(3)
    week_list = test_data['week_number'].tolist()
    y_pred = -1
    count = 0
    sum = 0
    for week_id in range(31,38):
        if week_id in week_list:
            count+=1
            tmp_row = test_data[test_data['week_number'] == week_id  ]
            sum+= int( tmp_row['total_claims'])
    if count > 0:
        y_pred = int(sum/count)
    else:
        max_week = max(week_list)
        tmp_row = test_data[test_data['week_number'] ==  max_week  ] 
        y_pre = tmp_row['total_claims']
    cur_row =  pd.DataFrame([[cur_uu_id, y_pred, 41]], columns=['uu_id', 'total_claims', 'week_number'] )
    res = pd.concat([res,cur_row] ,ignore_index = True)

In [38]:
res.to_csv("Nov21_submission_prediction_output.csv", index=False)